In [1]:
import requests
from bs4 import BeautifulSoup

html = requests.get('https://www.bestatter.de/verzeichnis/').text

soup = BeautifulSoup(html)

linkes = []

possible_links = soup.find_all('a')

for link in possible_links:
    if link.has_attr('href'):
        linkes.append(link.attrs['href'])
#        print (link.attrs['href'])

In [2]:
from pandas import DataFrame

df_link = DataFrame (linkes, columns = ['path'])
df_link = df_link.drop_duplicates()
df_link.shape

(67, 1)

In [3]:
url_top = 'https://www.bestatter.de'

df_combine = df_link[df_link['path'].str.contains('/verzeichnis/')]
df_combine = df_combine.drop(df_combine.index[0])
df_combine['domain'] = url_top
df_combine['url'] = df_combine['domain'] + df_combine['path']
df_combine.shape

(16, 3)

In [4]:
df_combine

,path,domain,url
17,/verzeichnis/baden-wuerttemberg/bestatter-in-a...,https://www.bestatter.de,https://www.bestatter.de/verzeichnis/baden-wue...
18,/verzeichnis/bayern/bestatter-in-aschaffenburg/,https://www.bestatter.de,https://www.bestatter.de/verzeichnis/bayern/be...
19,/verzeichnis/berlin/bestatter-in-berlin/,https://www.bestatter.de,https://www.bestatter.de/verzeichnis/berlin/be...
20,/verzeichnis/brandenburg/bestatter-in-brandenb...,https://www.bestatter.de,https://www.bestatter.de/verzeichnis/brandenbu...
21,/verzeichnis/bremen/bestatter-in-bremen/,https://www.bestatter.de,https://www.bestatter.de/verzeichnis/bremen/be...
22,/verzeichnis/hamburg/bestatter-in-hamburg/,https://www.bestatter.de,https://www.bestatter.de/verzeichnis/hamburg/b...
23,/verzeichnis/hessen/bestatter-in-darmstadt/,https://www.bestatter.de,https://www.bestatter.de/verzeichnis/hessen/be...
24,/verzeichnis/mecklenburg-vorpommern/bestatter-...,https://www.bestatter.de,https://www.bestatter.de/verzeichnis/mecklenbu...
25,/verzeichnis/niedersachsen/bestatter-in-brauns...,https://www.bestatter.de,https://www.bestatter.de/verzeichnis/niedersac...
26,/verzeichnis/nordrhein-westfalen/bestatter-in-...,https://www.bestatter.de,https://www.bestatter.de/verzeichnis/nordrhein...


In [5]:
import shutil
from os import makedirs

bundesland_local = 'bestatter/bundesland/'

makedirs(bundesland_local, exist_ok = True)
shutil.rmtree(bundesland_local)
makedirs(bundesland_local, exist_ok = True)

In [6]:
for index, row in df_combine.iterrows():
    
    url = row['url']
#    print('Downloading :', url)
    resp = requests.get(url)
    
    fname = bundesland_local + str(index) + '.html'
#    print('Saving: ', fname)
    
    with open(fname, 'w') as wf:
        wf.write(resp.text)

print('\n')
print('done...')



done...


In [7]:
from glob import glob
from os.path import join

html_filenames = glob(join(bundesland_local, '*.html'))

In [8]:
region_local = 'bestatter/region/'

makedirs(region_local, exist_ok = True)
shutil.rmtree(region_local)
makedirs(region_local, exist_ok = True)

In [9]:
from urllib.parse import urljoin

for hname in html_filenames:
    
    with open(hname, 'r') as rf:
        txt = rf.read()
        soup = BeautifulSoup(txt, 'lxml')
        
        for active in soup.find_all('ul', {'class': 'side-nav'}):
            active_link = active.find('li', {'class': 'active'})
            for each in active_link.find_all('li'):
                href = each.find('a').attrs['href']
                url = urljoin(url_top, href)
#                print('Downloading: ', url)
                resp = requests.get(url)

                fn = href.replace('/', '-').strip('-') + '.html'
                full_fname = join(region_local, fn)
#                print('Saving ', full_fname)
                with open(full_fname, 'w') as wf:
                    wf.write(resp.text)

print('\n')
print('done...')



done...


In [10]:
import re

regex = re.compile(r'[\n\r\t]')

import pandas as pd
import glob
import os.path

df_bestatter = pd.DataFrame()

count = 0

for file_name in glob.glob(os.path.join(region_local, '*.html')):
    with open(file_name) as html_file:
        soup = BeautifulSoup(html_file)
#        print (file_name)
        
        for i in soup.select('main'):
            cards_data = soup.find_all('li', attrs={'class', 'branchlist-item col-xl-4 col-lg-6 col-md-6 col-sm-6'})
#            print('Number of cards found       : ', len(cards_data))
            count += len(cards_data)
#            print('Total Number of cards found : ', count)
#            print('\n')
#            print(cards_data)
            
            for card in cards_data:
                company = card.find('h2', class_='branchlist-branch-company').text.strip()
                company = regex.sub(' ', company)
#                print(company)

                address = card.find('span', class_='branchlist-branch-address').text.strip()
#                print (address)
                city = card.find('span', class_='branchlist-branch-city').text.strip()
#                print (city)

                df_bestatter = df_bestatter.append(pd.DataFrame([[company, address, city]],
                                                                columns=['Marke', 'Strasse', 'Plz_Ort']))

print('\n')
print('Total Number of cards found : ', count)
print('done...')



Total Number of cards found :  2504
done...


In [11]:
df_bestatter.reset_index(drop = True, inplace = True)
#df_bestatter

In [12]:
df_bestatter = df_bestatter[df_bestatter['Marke'].notna()]
df_bestatter = df_bestatter.drop_duplicates(subset=['Marke', 'Plz_Ort'], keep='first')
#df_bestatter

In [13]:
df_bestatter['Plz'], df_bestatter['Ort'] = df_bestatter['Plz_Ort'].str.split(' ', 1).str
#df_bestatter

/Applications/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: Columnar iteration over characters will be deprecated in future releases.
  """Entry point for launching an IPython kernel.


In [14]:
df_bestatter['Source'] = 'https://www.bestatter.de/verzeichnis/'
#df_bestatter

In [15]:
df_bestatter = df_bestatter.drop('Plz_Ort',axis=1)

df_bestatter

,Marke,Strasse,Plz,Ort,Source
0,Bestattungsinstitut Huether-Heissler GmbH,Plittersdorfer Str. 2,76437,Rastatt,https://www.bestatter.de/verzeichnis/
1,Bestattungsinstitut Huether-Heissler GmbH,Wörtwiese 7,76479,Steinmauern,https://www.bestatter.de/verzeichnis/
2,Bestattungsinstitut Huether-Heissler GmbH,Kirschbaumallee 5,76448,Durmersheim,https://www.bestatter.de/verzeichnis/
3,Bestattungsinstitut Huether-Heissler GmbH,Hauptstr. 10,76287,Rheinstetten,https://www.bestatter.de/verzeichnis/
4,Bestattungs-Institut Sieglinde Berdon-Teuber I...,Kaiserstr. 55,76437,Rastatt,https://www.bestatter.de/verzeichnis/
...,...,...,...,...,...
2498,Bestattungen Hauck Inh. Torsten Hauck,Brunsbütteler Damm 258,13581,Berlin,https://www.bestatter.de/verzeichnis/
2499,Klaus und Sascha Kintrup GbR Bestattungen,Schalückstr. 10,33332,Gütersloh,https://www.bestatter.de/verzeichnis/
2500,Benkert e. K. Inh. Ursula Temme Bestattungsins...,Barthstr. 5-7,33330,Gütersloh,https://www.bestatter.de/verzeichnis/
2501,"Bestattungen Lohmann GbR Peter, Markus und Mat...",Friedhofstr. 33,33330,Gütersloh,https://www.bestatter.de/verzeichnis/


In [16]:
df_bestatter.to_csv('bestatter/df_bestatter_20200609.csv', sep = ';', index = False)

In [17]:
import pandas as pd

df = pd.read_csv('bestatter/df_bestatter_20200609.csv', sep = ';')
df

,Marke,Strasse,Plz,Ort,Source
0,Bestattungsinstitut Huether-Heissler GmbH,Plittersdorfer Str. 2,76437,Rastatt,https://www.bestatter.de/verzeichnis/
1,Bestattungsinstitut Huether-Heissler GmbH,Wörtwiese 7,76479,Steinmauern,https://www.bestatter.de/verzeichnis/
2,Bestattungsinstitut Huether-Heissler GmbH,Kirschbaumallee 5,76448,Durmersheim,https://www.bestatter.de/verzeichnis/
3,Bestattungsinstitut Huether-Heissler GmbH,Hauptstr. 10,76287,Rheinstetten,https://www.bestatter.de/verzeichnis/
4,Bestattungs-Institut Sieglinde Berdon-Teuber I...,Kaiserstr. 55,76437,Rastatt,https://www.bestatter.de/verzeichnis/
...,...,...,...,...,...
2098,Bestattungen Hauck Inh. Torsten Hauck,Brunsbütteler Damm 258,13581,Berlin,https://www.bestatter.de/verzeichnis/
2099,Klaus und Sascha Kintrup GbR Bestattungen,Schalückstr. 10,33332,Gütersloh,https://www.bestatter.de/verzeichnis/
2100,Benkert e. K. Inh. Ursula Temme Bestattungsins...,Barthstr. 5-7,33330,Gütersloh,https://www.bestatter.de/verzeichnis/
2101,"Bestattungen Lohmann GbR Peter, Markus und Mat...",Friedhofstr. 33,33330,Gütersloh,https://www.bestatter.de/verzeichnis/
